In [156]:
net = pp.networks.case39()

INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}


In [162]:
type(net.line.from_bus[0])

numpy.uint32

In [1]:
import pandapower as pp
import pandas as pd
import itertools

# Pandapower N-2 anaylsis code


def run_power_flow(net):
    pp.runpp(net)
    return net.res_bus, net.res_line


def get_line_buses(net, line_index):
    # Fetches the from_bus and to_bus of a line
    from_bus = net.line.at[line_index, "from_bus"] + 1
    to_bus = net.line.at[line_index, "to_bus"] + 1
    return (from_bus, to_bus)


def n_3_analysis(net):
    original_line_status = net.line.in_service.copy()
    contingency_results = []

    # Generate all combinations of three lines
    line_combinations = itertools.combinations(net.line.index, 3)

    for line1, line2, line3 in line_combinations:
        # Fetch bus information for the lines
        line1_buses = get_line_buses(net, line1)
        line2_buses = get_line_buses(net, line2)
        line3_buses = get_line_buses(net, line3)

        # Disconnect lines
        net.line.in_service.at[line1] = False
        net.line.in_service.at[line2] = False
        net.line.in_service.at[line3] = False

        # Run power flow
        try:
            pp.runpp(net)
            # Check if power flow converged
            converged = net["converged"]
            bus_voltages = net.res_bus.vm_pu.tolist()
            line_loadings = net.res_line.loading_percent.tolist()
            contingency_results.append(
                (
                    line1_buses,
                    line2_buses,
                    line3_buses,
                    converged,
                    bus_voltages,
                    line_loadings,
                )
            )
        except pp.LoadflowNotConverged:
            # Handle cases where power flow does not converge
            contingency_results.append(
                (line1_buses, line2_buses, line3_buses, False, [], [])
            )

        # Restore line status
        net.line.in_service.at[line1] = True
        net.line.in_service.at[line2] = True
        net.line.in_service.at[line3] = True

    return pd.DataFrame(
        contingency_results,
        columns=[
            "Line 1",
            "Line 2",
            "Line 3",
            "Converged",
            "Bus Voltages",
            "Line Loadings",
        ],
    )


def n_2_analysis(net):
    original_line_status = net.line.in_service.copy()
    contingency_results = []

    # Generate all combinations of two lines
    line_combinations = itertools.combinations(net.line.index, 2)

    for line1, line2 in line_combinations:
        # Fetch bus information for the lines
        line1_buses = get_line_buses(net, line1)
        line2_buses = get_line_buses(net, line2)

        # Disconnect lines
        net.line.in_service.at[line1] = False
        net.line.in_service.at[line2] = False

        # Run power flow
        try:
            pp.runpp(net)
            # Check if power flow converged
            converged = net["converged"]
            bus_voltages = net.res_bus.vm_pu.tolist()
            line_loadings = net.res_line.loading_percent.tolist()
            contingency_results.append(
                (line1_buses, line2_buses, converged, bus_voltages, line_loadings)
            )
        except pp.LoadflowNotConverged:
            # Handle cases where power flow does not converge
            contingency_results.append((line1_buses, line2_buses, False, [], []))

        # Restore line status
        net.line.in_service.at[line1] = True
        net.line.in_service.at[line2] = True

    return pd.DataFrame(
        contingency_results,
        columns=["Line 1", "Line 2", "Converged", "Bus Voltages", "Line Loadings"],
    )


# Create and setup network

net = pp.networks.case39()

# Run initial power flow
initial_bus_res, initial_line_res = run_power_flow(net)

initial_line_res

# Perform N-2 analysis
contingency_results = n_3_analysis(net)
print("\nN-2 Contingency Analysis Results:")
contingency_results

AttributeError: module 'pandapower' has no attribute 'networks'

In [155]:
contingency_results.loc[contingency_results["Converged"] == False]

,Line 1,Line 2,Converged,Slack Bus Voltage
0,"(0, 1)","(0, 38)",True,NaN
1,"(0, 1)","(1, 2)",True,NaN
2,"(0, 1)","(1, 24)",True,NaN
3,"(0, 1)","(2, 3)",True,NaN
4,"(0, 1)","(2, 17)",True,NaN
...,...,...,...,...
590,"(25, 26)","(25, 28)",True,NaN
591,"(25, 26)","(27, 28)",True,NaN
592,"(25, 27)","(25, 28)",True,NaN
593,"(25, 27)","(27, 28)",True,NaN


In [154]:
contingency_results.to_csv("contingency_results.csv", index=False)

In [5]:
# pyPSA n-2
# Standard Libraries
import os
import sys
from datetime import datetime, timezone, timedelta

# 3rd Party Libraries
import pypsa
import pandas as pd
import cmath
import matlab.engine
import pypsa
import pypower.api as pypower
import pandapower
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt

In [6]:
case_file = "../models/grid_models/IEEE_39_bus.RAW"

eng = matlab.engine.start_matlab()
eng.workspace["case_path"] = case_file
eng.eval("mpc = psse2mpc(case_path)", nargout=0)
eng.eval("savecase('here.mat',mpc,1.0)", nargout=0)

# Load the MATPOWER case file from a .mat file
ppc = pypower.loadcase("./here.mat")

# Convert Pypower network to PyPSA network
pypsa_network = pypsa.Network()
pypsa_network.import_from_pypower_ppc(ppc, overwrite_zero_s_nom=True)
pypsa_network.set_snapshots([datetime.now().strftime("%m/%d/%Y %H:%M:%S")])

pypsa_network.pf()

network = pypsa_network

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/19/2023 12:17:32'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.105964 seconds


In [81]:
import pandapower as pp
import pandapower.networks as nw
import pandas as pd
import itertools

net = nw.case39()

INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}


In [85]:
pandapower.toolbox.drop_lines(net, [0])

INFO:pandapower.toolbox:dropped 1 lines with 0 line switches


In [95]:
net.liones

,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,1557.117675,1.0,0,0.0,True,1.0,1.004087,100.0,None,1,4.165875,None,1,ol,48.919275
1,1671.444477,1.0,0,0.0,True,1.0,1.673479,100.0,None,1,1.190250,None,38,ol,29.756250
2,573.194026,1.0,1,0.0,True,1.0,0.836740,100.0,None,1,1.547325,None,2,ol,17.972775
3,325.374525,1.0,1,0.0,True,1.0,0.836740,100.0,None,1,8.331750,None,24,ol,10.236150
4,493.410410,1.0,2,0.0,True,1.0,0.836740,100.0,None,1,1.547325,None,3,ol,25.352325
5,476.473106,1.0,2,0.0,True,1.0,0.836740,100.0,None,1,1.309275,None,17,ol,15.830325
6,299.077132,1.0,3,0.0,True,1.0,1.004087,100.0,None,1,0.952200,None,4,ol,15.235200
7,307.991502,1.0,3,0.0,True,1.0,0.836740,100.0,None,1,0.952200,None,13,ol,15.354225
8,96.720920,1.0,4,0.0,True,1.0,2.008175,100.0,None,1,0.238050,None,5,ol,3.094650
9,328.940273,1.0,4,0.0,True,1.0,1.506131,100.0,None,1,0.952200,None,7,ol,13.330800


In [99]:
net = nw.case39()

INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}


In [106]:
net.line

,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,1557.117675,1.0,0,0.0,True,1.0,1.004087,100.0,None,1,4.165875,None,1,ol,48.919275
1,1671.444477,1.0,0,0.0,True,1.0,1.673479,100.0,None,1,1.190250,None,38,ol,29.756250
2,573.194026,1.0,1,0.0,True,1.0,0.836740,100.0,None,1,1.547325,None,2,ol,17.972775
3,325.374525,1.0,1,0.0,True,1.0,0.836740,100.0,None,1,8.331750,None,24,ol,10.236150
4,493.410410,1.0,2,0.0,True,1.0,0.836740,100.0,None,1,1.547325,None,3,ol,25.352325
5,476.473106,1.0,2,0.0,True,1.0,0.836740,100.0,None,1,1.309275,None,17,ol,15.830325
6,299.077132,1.0,3,0.0,True,1.0,1.004087,100.0,None,1,0.952200,None,4,ol,15.235200
7,307.991502,1.0,3,0.0,True,1.0,0.836740,100.0,None,1,0.952200,None,13,ol,15.354225
8,96.720920,1.0,4,0.0,True,1.0,2.008175,100.0,None,1,0.238050,None,5,ol,3.094650
9,328.940273,1.0,4,0.0,True,1.0,1.506131,100.0,None,1,0.952200,None,7,ol,13.330800


In [111]:
import pandapower
import itertools

# Assuming 'net' is your PandaPower network
line_indices = net.line.index.tolist()

# Generate all unique combinations of two lines
n_2_combinations = list(itertools.combinations(line_indices, 2))


for (
    line1,
    line2,
) in range(n_2_combinations):
    print(line1, line2)

TypeError: 'list' object cannot be interpreted as an integer

In [114]:
for elm in range(n_2_combinations):
    elm

TypeError: 'list' object cannot be interpreted as an integer

In [109]:
n_2_combinations

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (1, 30),
 (1, 31),
 (1, 32),
 (1, 33),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (2, 19),
 (2, 20),
 (2, 21),
 (2, 22),
 (2, 23),
 (2, 24),
 (2, 25),
 (2, 26),
 (2, 27),
 (2, 28),
 (2, 29),
 (2, 30),
 (2, 31),
 (2, 32),
 (2, 33),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),


In [107]:
unique_lines

[(0, 1),
 (0, 38),
 (1, 2),
 (1, 24),
 (2, 3),
 (2, 17),
 (3, 4),
 (3, 13),
 (4, 5),
 (4, 7),
 (5, 6),
 (5, 10),
 (6, 7),
 (7, 8),
 (8, 38),
 (9, 10),
 (9, 12),
 (12, 13),
 (13, 14),
 (14, 15),
 (15, 16),
 (15, 18),
 (15, 20),
 (15, 23),
 (16, 17),
 (16, 26),
 (20, 21),
 (21, 22),
 (22, 23),
 (22, 35),
 (24, 25),
 (25, 26),
 (25, 27),
 (25, 28),
 (27, 28)]

In [117]:
import pandapower as pp
import pandapower.networks as nw
import pandas as pd
import itertools


import pandapower
import itertools


# Load the network
net = nw.case39()

# Define new columns for the DataFrame
columns = ["Line 1", "Line 2", "Result", "Iterations"]
contingency_df = pd.DataFrame(columns=columns)


# Assuming 'net' is your PandaPower network
line_indices = net.line.index.tolist()

# Generate all unique combinations of two lines
n_2_combinations = itertools.combinations(line_indices, 2)

# Iterate over each combination
for combo in n_2_combinations:
    line1, line2 = combo  # Unpack the combination

    # Perform your desired operations with line1 and line2
    # For example, you can drop these lines from a copy of the network
    # and perform simulations or analyses
    net = nw.case39()

    pandapower.toolbox.drop_lines(net, [line1, line2])

    try:
        # Run a power flow
        pp.runpp(net, algorithm="nr", max_iteration=100)

        # Check the power flow results
        converged = net.converged
        iterations = net._ppc["iterations"]

    except pp.LoadflowNotConverged:
        # Handle cases where power flow doesn't converge
        converged = False
        iterations = None

    # Append the results to the DataFrame
    contingency_df = contingency_df.append(
        {
            "Line 1": line1,
            "Line 2": line2,
            "Result": converged,  # Use 'converged' to indicate success or failure
            "Iterations": iterations,
        },
        ignore_index=True,
    )

INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}
INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}
INFO:pandapower.toolbox:dropped 2 lines with 0 line switches


True


INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}
INFO:pandapower.toolbox:dropped 2 lines with 0 line switches


True


INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}
INFO:pandapower.toolbox:dropped 2 lines with 0 line switches


...

True


INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}
INFO:pandapower.toolbox:dropped 2 lines with 0 line switches


True


INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}


KeyboardInterrupt: 

In [105]:
contingency_df

,Line 1,Line 2,Result,Iterations
0,0,NaN,True,3
1,0,NaN,True,3
2,1,NaN,True,3
3,1,NaN,True,3
4,2,NaN,True,3
5,2,NaN,True,3
6,3,NaN,True,3
7,3,NaN,True,3
8,4,NaN,True,3
9,4,NaN,True,3


In [91]:
unique_lines[0][0]

0

In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandas as pd
import itertools

# Load the network
net = nw.case39()

In [4]:
net.links

AttributeError: 'pandapowerNet' instance has no attribute 'links'

In [ ]:
import pandapower as pp
import pandapower.networks as nw
import pandas as pd
import itertools

# Load the network
net = nw.case39()

# Define new columns for the DataFrame
columns = ["Line 1", "Line 2", "Result", "Iterations"]
contingency_df = pd.DataFrame(columns=columns)

from_bus = net.line.from_bus.to_list()
to_bus = net.line.to_bus.to_list()
unique_lines = list(zip(from_bus, to_bus))

for i in range(len(unique_lines)):
    for j in range(i + 1, len(unique_lines)):
        line1 = unique_lines[i]
        line2 = unique_lines[j]

        # Save original in_service status
        original_status_line1 = net.line.loc[
            (net.line.from_bus == line1[0]) & (net.line.to_bus == line1[1]),
            "in_service",
        ]
        original_status_line2 = net.line.loc[
            (net.line.from_bus == line2[0]) & (net.line.to_bus == line2[1]),
            "in_service",
        ]

        # Outage both lines
        net.line.loc[
            (net.line.from_bus == line1[0]) & (net.line.to_bus == line1[1]),
            "in_service",
        ] = False
        net.line.loc[
            (net.line.from_bus == line2[0]) & (net.line.to_bus == line2[1]),
            "in_service",
        ] = False

        pandapower.toolbox.drop_lines(
            net,
        )

        try:
            # Run a power flow
            pp.runpp(net, algorithm="nr", max_iteration=100)

            # Check the power flow results
            converged = net.converged
            iterations = net._ppc["iterations"]

        except pp.LoadflowNotConverged:
            # Handle cases where power flow doesn't converge
            converged = False
            iterations = None

        # Append the results to the DataFrame
        contingency_df = contingency_df.append(
            {
                "Line 1": line1,
                "Line 2": line2,
                "Result": converged,  # Use 'converged' to indicate success or failure
                "Iterations": iterations,
            },
            ignore_index=True,
        )

        # Restore original in_service status

        net.line.loc[
            (net.line.from_bus == line1[0]) & (net.line.to_bus == line1[1]),
            "in_service",
        ] = original_status_line1
        net.line.loc[
            (net.line.from_bus == line2[0]) & (net.line.to_bus == line2[1]),
            "in_service",
        ] = original_status_line2

INFO:pandapower.convert_format:These dtypes could not be corrected: {'trafo': ['tap_min', 'tap_max']}


In [8]:
network.links

attribute,bus0,bus1,type,carrier,efficiency,build_year,lifetime,p_nom,p_nom_extendable,p_nom_min,...,p_set,p_min_pu,p_max_pu,capital_cost,marginal_cost,length,terrain_factor,ramp_limit_up,ramp_limit_down,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,,,


In [165]:
# pyPSA n-2
from_bus = network.lines["bus0"].to_list()
to_bus = network.lines["bus1"].to_list()
unique_lines = list(zip(from_bus, to_bus))

# Define new columns for the DataFrame
columns = ["Line 1", "Line 2", "Result", "Iterations"]
contingency_df = pd.DataFrame(columns=columns)

for i in range(len(unique_lines)):
    for j in range(i + 1, len(unique_lines)):

        temp_network = network.copy()

        line1 = unique_lines[i]
        line2 = unique_lines[j]

        # Save original in_service status
        original_status_line1 = network.lines.loc[
            (temp_network.lines["bus0"] == line1[0])
            & (temp_network.lines["bus1"] == line1[1]),
            "status",
        ]
        original_status_line2 = network.lines.loc[
            (temp_network.lines["bus0"] == line2[0])
            & (temp_network.lines["bus1"] == line2[1]),
            "status",
        ]

        # Outage both lines
        temp_network.lines.loc[
            (temp_network.lines["bus0"] == line1[0])
            & (temp_network.lines["bus1"] == line1[1]),
            "status",
        ] = 0.0
        temp_network.lines.loc[
            (temp_network.lines["bus0"] == line2[0])
            & (temp_network.lines["bus1"] == line2[1]),
            "status",
        ] = 0.0

        temp_network.snapshots = [datetime.now().strftime("%m/%d/%Y %H:%M:%S")]

        temp = temp_network.pf(temp_network.snapshots[0])
        result = temp["converged"]["0"].values[0]
        itr = temp["n_iter"]["0"].values[0]
        error = temp["error"]["0"].values[0]

        # Append the results to the DataFrame
        contingency_df = contingency_df.append(
            {
                "Line 1": line1,
                "Line 2": line2,
                "Result": result,  # Use 'converged' to indicate success or failure
                "Iterations": itr,
            },
            ignore_index=True,
        )

        # Outage both lines
        temp_network.lines.loc[
            (temp_network.lines["bus0"] == line1[0])
            & (temp_network.lines["bus1"] == line1[1]),
            "status",
        ] = original_status_line1
        temp_network.lines.loc[
            (temp_network.lines["bus0"] == line2[0])
            & (temp_network.lines["bus1"] == line2[1]),
            "status",
        ] = original_status_line2

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/14/2023 16:25:16'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.083971 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/14/2023 16:25:17'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.077976 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/14/2023 16:25:19'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.080961 seconds
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/14/2023 16:25:20'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.075258 seconds


KeyboardInterrupt: 

In [4]:
network.snapshots

Index(['12/15/2023 14:47:09'], dtype='object', name='snapshot')

In [161]:
network.lines.loc[
    (network.lines["bus0"] == "1") & (network.lines["bus1"] == "2"), "status"
] = 0.0
network.lines.loc[
    (network.lines["bus0"] == "1") & (network.lines["bus1"] == "2"), "status"
]

Line
L0    0.0
Name: status, dtype: float64

In [167]:
network.sn

In [168]:
network

PyPSA Network
Components:
 - Bus: 39
 - Generator: 10
 - Line: 35
 - Load: 19
 - ShuntImpedance: 2
 - SubNetwork: 1
 - Transformer: 11
Snapshots: 1

In [144]:
len(network.lines)

35

In [145]:
network.lines.status.count()

35

In [152]:
network.lines.status[0]

0.0

In [149]:
network.lines.loc[
    (network.lines["bus0"] == "1") & (network.lines["bus1"] == "2"), "status"
]

Line
L0    0.0
Name: status, dtype: float64

In [129]:
network.lines.status.count()

35

In [133]:
network.lines.status[0]

0.0

In [122]:
temp = network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/14/2023 15:46:33'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.097971 seconds


In [116]:
temp

{'n_iter': SubNetwork           0
 snapshot              
 12/14/2023 15:46:33  4,
 'error': SubNetwork                      0
 snapshot                         
 12/14/2023 15:46:33  2.078536e-09,
 'converged': SubNetwork              0
 snapshot                 
 12/14/2023 15:46:33  True}

In [119]:
temp = network.pf()
result = temp["converged"]["0"].values[0]
itr = temp["n_iter"]["0"].values[0]
error = temp["error"]["0"].values[0]

True

In [120]:
itr = temp["n_iter"]["0"].values[0]

4

In [95]:
network.lines

,bus0,bus1,r,x,b,s_nom,rateB,rateC,status,v_ang_min,...,num_parallel,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
Line,,,,,,,,,,,,,,,,,,,,,
L0,1,2,0.000035,0.000411,69.87,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000411,0.000035,0.0,69.87,0.000411,0.000035,0.0,1.0
L1,1,39,0.000010,0.000250,75.00,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000250,0.000010,0.0,75.00,0.000250,0.000010,0.0,1.0
L2,2,3,0.000013,0.000151,25.72,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000151,0.000013,0.0,25.72,0.000151,0.000013,0.0,1.0
L3,2,25,0.000070,0.000086,14.60,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000086,0.000070,0.0,14.60,0.000086,0.000070,0.0,1.0
L4,3,4,0.000013,0.000213,22.14,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000213,0.000013,0.0,22.14,0.000213,0.000013,0.0,1.0
L5,3,18,0.000011,0.000133,21.38,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000133,0.000011,0.0,21.38,0.000133,0.000011,0.0,1.0
L6,4,5,0.000008,0.000128,13.42,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000128,0.000008,0.0,13.42,0.000128,0.000008,0.0,1.0
L7,4,14,0.000008,0.000129,13.82,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000129,0.000008,0.0,13.82,0.000129,0.000008,0.0,1.0
L8,5,6,0.000002,0.000026,4.34,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000026,0.000002,0.0,4.34,0.000026,0.000002,0.0,1.0


In [102]:
network.lines.iloc[0].bus0

'1'

In [105]:
# Outage both lines
network.lines.loc[
    (network.lines["bus0"] == "1") & (network.lines["bus1"] == "2"), "status"
] = 0

In [49]:
from_bus = net.line.from_bus.to_list()
to_bus = net.line.to_bus.to_list()
unique_lines = list(zip(from_bus, to_bus))
unique_lines

[(0, 1),
 (0, 38),
 (1, 2),
 (1, 24),
 (2, 3),
 (2, 17),
 (3, 4),
 (3, 13),
 (4, 5),
 (4, 7),
 (5, 6),
 (5, 10),
 (6, 7),
 (7, 8),
 (8, 38),
 (9, 10),
 (9, 12),
 (12, 13),
 (13, 14),
 (14, 15),
 (15, 16),
 (15, 18),
 (15, 20),
 (15, 23),
 (16, 17),
 (16, 26),
 (20, 21),
 (21, 22),
 (22, 23),
 (22, 35),
 (24, 25),
 (25, 26),
 (25, 27),
 (25, 28),
 (27, 28)]

In [47]:
line1 = unique_lines[0]
line2 = unique_lines[1]

In [54]:
net.line.loc[
    (net.line.from_bus == line1[0]) & (net.line.to_bus == line1[1]), "in_service"
] = False

In [66]:
contingency_df.loc[contingency_df.Result == False]

,Line 1,Line 2,Result,Iterations
270,"(4, 7)","(5, 6)",False,None


In [62]:
contingency_df[
    (contingency_df["Line 1"] == (0, 1)) & (contingency_df["Line 2"] == (0, 38))
]

,Line 1,Line 2,Result,Iterations
0,"(0, 1)","(0, 38)",True,3


In [5]:
import pypsa
import numpy as np

In [6]:
network = pypsa.Network()

In [7]:
n_buses = 3

for i in range(n_buses):
    network.add("Bus", "My bus {}".format(i), v_nom=20.0)

network.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
My bus 0,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 1,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 2,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,


In [8]:
for i in range(n_buses):
    network.add(
        "Line",
        "My line {}".format(i),
        bus0="My bus {}".format(i),
        bus1="My bus {}".format((i + 1) % n_buses),
        x=0.1,
        r=0.01,
    )

network.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
My line 0,My bus 0,My bus 1,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
My line 1,My bus 1,My bus 2,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
My line 2,My bus 2,My bus 0,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
network.add("Generator", "My gen", bus="My bus 0", p_set=100, control="PQ")

network.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
My gen,My bus 0,PQ,,0.0,False,0.0,inf,0.0,1.0,100.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [10]:
network.generators.p_set

Generator
My gen    100.0
Name: p_set, dtype: float64

In [11]:
network.add("Load", "My load", bus="My bus 1", p_set=100)

network.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
My load,My bus 1,,,100.0,0.0,-1.0


In [12]:
network.loads.p_set

Load
My load    100.0
Name: p_set, dtype: float64

In [27]:
network.loads.q_set = 100.0

In [41]:
network

PyPSA Network
Components:
 - Bus: 39
 - Generator: 10
 - Line: 35
 - Load: 19
 - ShuntImpedance: 2
 - SubNetwork: 1
 - Transformer: 11
Snapshots: 1

In [42]:
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/15/2023 15:00:29'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.052982 seconds


{'n_iter': SubNetwork           0
 snapshot              
 12/15/2023 15:00:29  4,
 'error': SubNetwork                      0
 snapshot                         
 12/15/2023 15:00:29  2.078423e-09,
 'converged': SubNetwork              0
 snapshot                 
 12/15/2023 15:00:29  True}

In [57]:
network.lines.iloc[0].status == 0

False

In [64]:
network.lines

,bus0,bus1,r,x,b,s_nom,rateB,rateC,status,v_ang_min,...,num_parallel,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
Line,,,,,,,,,,,,,,,,,,,,,
L0,1,2,0.000035,0.000411,69.87,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000411,0.000035,0.0,69.87,0.000411,0.000035,0.0,1.0
L1,1,39,0.000010,0.000250,75.00,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000250,0.000010,0.0,75.00,0.000250,0.000010,0.0,1.0
L2,2,3,0.000013,0.000151,25.72,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000151,0.000013,0.0,25.72,0.000151,0.000013,0.0,1.0
L3,2,25,0.000070,0.000086,14.60,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000086,0.000070,0.0,14.60,0.000086,0.000070,0.0,1.0
L4,3,4,0.000013,0.000213,22.14,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000213,0.000013,0.0,22.14,0.000213,0.000013,0.0,1.0
L5,3,18,0.000011,0.000133,21.38,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000133,0.000011,0.0,21.38,0.000133,0.000011,0.0,1.0
L6,4,5,0.000008,0.000128,13.42,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000128,0.000008,0.0,13.42,0.000128,0.000008,0.0,1.0
L7,4,14,0.000008,0.000129,13.82,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000129,0.000008,0.0,13.82,0.000129,0.000008,0.0,1.0
L8,5,6,0.000002,0.000026,4.34,1.0,0.0,1.0,1.0,-360.0,...,1.0,0,0.000026,0.000002,0.0,4.34,0.000026,0.000002,0.0,1.0


In [69]:
network.lines.loc[
    (network.lines["bus0"] == "1") & (network.lines["bus1"] == "2"), "status"
]

Line
L0    0.0
Name: status, dtype: float64

In [75]:
network.lines.status = 0.0

In [79]:
network.lines.status

Line
L0     0.0
L1     0.0
L2     0.0
L3     0.0
L4     0.0
L5     0.0
L6     0.0
L7     0.0
L8     0.0
L9     0.0
L10    0.0
L11    0.0
L12    0.0
L13    0.0
L14    0.0
L15    0.0
L16    0.0
L17    0.0
L18    0.0
L19    0.0
L20    0.0
L21    0.0
L22    0.0
L23    0.0
L24    0.0
L25    0.0
L26    0.0
L27    0.0
L28    0.0
L29    0.0
L30    0.0
L31    0.0
L32    0.0
L33    0.0
L34    0.0
Name: status, dtype: float64

In [76]:
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['12/15/2023 15:00:29'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.062980 seconds


{'n_iter': SubNetwork           0
 snapshot              
 12/15/2023 15:00:29  4,
 'error': SubNetwork                      0
 snapshot                         
 12/15/2023 15:00:29  2.078423e-09,
 'converged': SubNetwork              0
 snapshot                 
 12/15/2023 15:00:29  True}

In [58]:
network.lines.iloc[0].status

1.0

In [ ]:
network.lines.loc[
    (network.lines.from_bus == line1[0]) & (net.line.to_bus == line1[1]), "in_service"
] = False

In [46]:
network.lines_t.p0

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,L25,L26,L27,L28,L29,L30,L31,L32,L33,L34
snapshot,,,,,,,,,,,,,,,,,,,,,
12/15/2023 15:00:29,-121.096028,121.096028,364.305679,-235.86942,83.606861,-42.94271,-149.70646,-266.870935,-468.387463,318.498956,...,15.762179,-590.406669,56.951944,340.994025,74.355227,266.212581,-140.808317,-190.209503,-347.5964,-558.573239


In [47]:
network.buses_t.v_ang * 180 / np.pi

Bus,1,2,3,4,5,6,7,8,9,10,...,30,31,32,33,34,35,36,37,38,39
snapshot,,,,,,,,,,,,,,,,,,,,,
12/15/2023 15:00:29,-8.368513,-5.771915,-8.608104,-9.512179,-8.432428,-7.733848,-9.907067,-10.402832,-10.165247,-5.30599,...,-3.359887,0.0,2.658,3.995414,2.981283,5.208169,7.821122,2.393336,7.71561,-9.93535


In [48]:
network.buses_t.v_mag_pu

Bus,1,2,3,4,5,6,7,8,9,10,...,30,31,32,33,34,35,36,37,38,39
snapshot,,,,,,,,,,,,,,,,,,,,,
12/15/2023 15:00:29,1.048516,1.051598,1.036419,1.016848,1.013999,1.0109,1.001945,1.001797,1.030602,1.021327,...,1.048,0.982,0.983,0.997,1.012,1.049,1.064,1.028,1.027,1.03


In [49]:
network

PyPSA Network
Components:
 - Bus: 39
 - Generator: 10
 - Line: 35
 - Load: 19
 - ShuntImpedance: 2
 - SubNetwork: 1
 - Transformer: 11
Snapshots: 1